In [2]:
import pandas as pd

df = pd.read_csv("./data.csv")
dataset = df["Sentence"].tolist()
print(dataset)

["The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .", '$ESI on lows, down $1.50 to $2.50 BK a real possibility', "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .", 'According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .', 'The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .', "$SPY wouldn't be surprised to see a green close", "Shell's $70 Billion BG Deal Meets Shareholder Skepticism", 'SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANGE RELEASE OCTOBER 14 , 2008 AT 2:45 PM The Company updates its full year outlook and estimat

In [7]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

# # Sample dataset
# dataset = ["I love playing football on the weekends",
#            "I enjoy hiking and camping in the mountains",
#            "I like to read books and watch movies",
#            "I prefer playing video games over sports",
#            "I love listening to music and going to concerts"]

# Vectorize the dataset
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset)

# Define the number of clusters
k = 5

# Create a k-means model and fit it to the data
km = KMeans(n_clusters=k)
km.fit(X)

# Predict the clusters for each document
y_pred = km.predict(X)

# Print the cluster assignments
print(y_pred)
# Output: [1 1 0 1 0]

pred_df = pd.DataFrame(y_pred, columns=["cluster"])
print(pred_df)

final_df = pd.concat([df, pred_df], axis=1)
print(final_df)

final_df.to_csv("./clustered_data.csv")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[3 3 2 ... 0 3 3]
      cluster
0           3
1           3
2           2
3           0
4           0
...       ...
5837        3
5838        3
5839        0
5840        3
5841        3

[5842 rows x 1 columns]
                                               Sentence Sentiment  cluster
0     The GeoSolutions technology will leverage Bene...  positive        3
1     $ESI on lows, down $1.50 to $2.50 BK a real po...  negative        3
2     For the last quarter of 2010 , Componenta 's n...  positive        2
3     According to the Finnish-Russian Chamber of Co...   neutral        0
4     The Swedish buyout firm has sold its remaining...   neutral        0
...                                                 ...       ...      ...
5837  RISING costs have forced packaging producer Hu...  negative        3
5838  Nordic Walking was first used as a summer trai...   neutral        3
5839  According shipping company Viking Line , the E...   neutral        0
5840  In the building and home improvem

In [38]:
import numpy as np

# from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.semi_supervised import LabelSpreading, SelfTrainingClassifier

# Loading dataset containing first five categories
# data = fetch_20newsgroups(
#     subset="train",
#     categories=[
#         "alt.atheism",
#         "comp.graphics",
#         "comp.os.ms-windows.misc",
#         "comp.sys.ibm.pc.hardware",
#         "comp.sys.mac.hardware",
#     ],
# )

import pandas as pd

data_df = pd.read_csv("./labeled_data.csv - clustered_data (1).csv")

feature = data_df["Sentence"].tolist()
target = data_df["manual_cluster"].tolist()
data = data_df.to_dict()

print("GASDASD", data)
# print("%d documents" % len(data.filenames))
# print("%d categories" % len(data.target_names))
# print()

# Parameters
sdg_params = dict(alpha=1e-5, penalty="l2", loss="log_loss")
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SGDClassifier(**sdg_params)),
    ]
)
# SelfTraining Pipeline
st_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SelfTrainingClassifier(SGDClassifier(**sdg_params), verbose=True)),
    ]
)
# LabelSpreading Pipeline
ls_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        # LabelSpreading does not support dense matrices
        ("toarray", FunctionTransformer(lambda x: x.toarray())),
        ("clf", LabelSpreading()),
    ]
)


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:", sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(
        "Micro-averaged F1 score on test set: %0.3f"
        % f1_score(y_test, y_pred, average="micro")
    )
    print("-" * 10)
    print()

    print("SDF", clf.predict(["B plans on reducing it's contribution"]))


if __name__ == "__main__":
    X, y = feature, [int(item) for item in target]
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # print("Supervised SGDClassifier on 100% of the data:")
    # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

    # select a mask of 20% of the train dataset
    y_mask = np.random.rand(len(y_train)) < 0.2

    # # X_20 and y_20 are the subset of the train dataset indicated by the mask
    # X_20, y_20 = map(
    #     list, zip(*((x, y) for x, y, m in zip(X_train, y_train, y_mask) if m))
    # )
    # print("Supervised SGDClassifier on 20% of the training data:")
    # eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

    # set the non-masked subset to be unlabeled
    # y_train[~y_mask] = -1
    print("SelfTrainingClassifier on 20% of the training data (rest is unlabeled):")
    eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)

    print("LabelSpreading on 20% of the data (rest is unlabeled):")
    eval_and_print_metrics(ls_pipeline, X_train, y_train, X_test, y_test)

GASDASD {'Unnamed: 0': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 1

In [56]:
print(data_df["Sentiment"].value_counts())

pos = data_df[data_df["Sentiment"] == "positive"].sample(frac=1)[:860]
neu = data_df[data_df["Sentiment"] == "neutral"].sample(frac=1)[:860]
neg = data_df[data_df["Sentiment"] == "negative"].sample(frac=1)[:860]

balanced_df = pd.concat([pos, neu, neg])

print(balanced_df)

balanced_df.to_csv("./balanced_data.csv")

neutral     3130
positive    1852
negative     860
Name: Sentiment, dtype: int64
      Unnamed: 0                                           Sentence Sentiment  \
1317        1317  Finnish steel maker Rautaruukki Oyj ( Ruukki )...  positive   
2061        2061  The financial impact is estimated to be an ann...  positive   
2426        2426  `` Lining stone sales were also good in the ea...  positive   
5450        5450  UPDATE 1-AstraZeneca sells rare cancer drug to...  positive   
1973        1973                           @pierrules: $SIRI LONG!!  positive   
...          ...                                                ...       ...   
834          834  The current reference grade instruments are ch...  negative   
125          125  The operating margin of Aker Yards Cruise & Fe...  negative   
5629        5629  Diluted earnings per share ( EPS ) declined to...  negative   
3214        3214  $QCOR a little pullback is fine but if this er...  negative   
2460        2460  Operating 

In [61]:
pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10